In [1]:
import os
#os.chdir('/Users/rupesh.karn/Desktop/WorkPart-1/UNSW-NB15 Dataset')
import pandas as pd
import numpy as np
import csv
import os
import time
from matplotlib.pyplot import *
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (16.0, 5.0)

# Read in the training CSV file
print("Reading Training csv file.")
df1 = pd.read_csv("/root/pathint/fig_split_mnist/UNSW_NB15_training-set.csv")
df1.drop('label', axis=1, inplace=True)

obj_df=df1

obj_df["proto"] = obj_df["proto"].astype('category')
obj_df["service"] = obj_df["service"].astype('category')
obj_df["state"] = obj_df["state"].astype('category')
obj_df["proto_cat"] = obj_df["proto"].cat.codes
obj_df["service_cat"] = obj_df["service"].cat.codes
obj_df["state_cat"] = obj_df["state"].cat.codes

obj_df["proto"] = obj_df["proto_cat"]
obj_df["service"] = obj_df["service_cat"]
obj_df["state"] = obj_df["state_cat"]

obj_df.drop('proto_cat', axis=1, inplace=True)
obj_df.drop('service_cat', axis=1, inplace=True)
obj_df.drop('state_cat', axis=1, inplace=True)

Y_train_all_attacks = obj_df["attack_cat"]
obj_df=pd.get_dummies(obj_df, columns=["attack_cat"])


X_train = obj_df.values[:,:-10]
Y_train_onehot_encoded = obj_df.values[:,-10:]

for j in range(0,43):
    maximum = max(X_train[:,j])
    for i in range(0,len(X_train)):
        X_train[i,j] = round(X_train[i,j]/maximum,3)

# Read in the testing CSV file 
print("Reading Testing csv file.")
df2 = pd.read_csv("/root/pathint/fig_split_mnist/UNSW_NB15_testing-set.csv")
df2.drop('label', axis=1, inplace=True)

obj_df2=df2

obj_df2["proto"] = obj_df2["proto"].astype('category')
obj_df2["service"] = obj_df2["service"].astype('category')
obj_df2["state"] = obj_df2["state"].astype('category')
obj_df2["proto_cat"] = obj_df2["proto"].cat.codes
obj_df2["service_cat"] = obj_df2["service"].cat.codes
obj_df2["state_cat"] = obj_df2["state"].cat.codes

obj_df2["proto"] = obj_df2["proto_cat"]
obj_df2["service"] = obj_df2["service_cat"]
obj_df2["state"] = obj_df2["state_cat"]

obj_df2.drop('proto_cat', axis=1, inplace=True)
obj_df2.drop('service_cat', axis=1, inplace=True)
obj_df2.drop('state_cat', axis=1, inplace=True)

Y_test_all_attacks = obj_df2["attack_cat"]
obj_df2=pd.get_dummies(obj_df2, columns=["attack_cat"])


X_test = obj_df2.values[:,:-10]
Y_test_onehot_encoded = obj_df2.values[:,-10:]

for j in range(0,43):
    maximum = max(X_train[:,j])
    for i in range(0,len(X_test)):
        X_test[i,j] = round(X_test[i,j]/maximum,3)


estimators_number = list(range(10,30))

dataspace = 0;
overall_accuracy_matrix = [None]*len(X_train)
iTERATION=0
dataspace_number=1
attack_type = 4
Y_train = obj_df.values[:,-attack_type]
Y_test = obj_df2.values[:,-attack_type]

cleanup_nums = {"Worms":0, "Shellcode":1, "Reconnaissance":2, "Normal":3, "Generic":4, "Fuzzers":5, "Exploits":6, "DoS":7, "Backdoor":8, "Analysis":9}
Y_train_all_attacks.replace(cleanup_nums,inplace=True)
Y_test_all_attacks.replace(cleanup_nums,inplace=True)

Reading Training csv file.
Reading Testing csv file.


In [2]:
from keras.utils import np_utils

#task_labels = [[0,1], [2,3], [4,5], [6,7], [8,9],[1,5],[7,9],[3,8],[0,6],[4,2]]
#task_labels = [[4,2], [0,6], [3,8], [9,7], [1,5],[8,9],[6,7],[5,5],[3,2],[0,1]]
#task_labels = [[8,9], [6,7], [4,5], [2,3], [0,1]]
#task_labels = [[0,9], [7,8], [3,6], [1,4], [2,5]]
#task_labels = [[0,1], [2,3],[4,5], [6,7],[8,9]]
task_labels = [[4,5], [8,9], [6,7], [0,1], [2,3]]
#task_labels = [[0,1], [2,3,1,0],[4,5,1,2], [6,7,3,0],[8,9,4,6]]
n_tasks = len(task_labels)
nb_classes  = 10
training_datasets = []
validation_datasets = []
multihead=False

for labels in task_labels:
    idx = np.in1d(Y_train_all_attacks, labels)
    if multihead:
        label_map = np.arange(nb_classes)
        label_map[labels] = np.arange(len(labels))
        data = X_train[idx], np_utils.to_categorical(label_map[Y_train_all_attacks[idx]], len(labels))
    else:
        data = X_train[idx], np_utils.to_categorical(Y_train_all_attacks[idx], nb_classes)
        training_datasets.append(data)

for labels in task_labels:
    idx = np.in1d(Y_test_all_attacks, labels)
    if multihead:
        label_map = np.arange(nb_classes)
        label_map[labels] = np.arange(len(labels))
        data = X_test[idx], np_utils.to_categorical(label_map[Y_test_all_attacks[idx]], len(labels))
    else:
        data = X_test[idx], np_utils.to_categorical(Y_test_all_attacks[idx], nb_classes)
        validation_datasets.append(data)
        
tasks_train={}; labels_train = {}; tasks_test = {}; labels_test = {}

for i in range(len(task_labels)):
    tasks_train[str(i)] = training_datasets[i][0]
    labels_train[str(i)] = training_datasets[i][1]
    tasks_test[str(i)] = validation_datasets[i][0]
    labels_test[str(i)] = validation_datasets[i][1]
    print('Task {0} size: Trainset - {1}, {2}, Testset - {3}, {4}'.format(i,tasks_train[str(i)].shape, labels_train[str(i)].shape, tasks_test[str(i)].shape, labels_test[str(i)].shape))

Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

Task 0 size: Trainset - (24933, 43), (24933, 10), Testset - (58184, 43), (58184, 10)
Task 1 size: Trainset - (1260, 43), (1260, 10), Testset - (3746, 43), (3746, 10)
Task 2 size: Trainset - (15221, 43), (15221, 10), Testset - (45657, 43), (45657, 10)
Task 3 size: Trainset - (422, 43), (422, 10), Testset - (1263, 43), (1263, 10)
Task 4 size: Trainset - (40496, 43), (40496, 10), Testset - (66491, 43), (66491, 10)


In [3]:
#Task labels sanity check for train set. Each row must has only one '1' 
print('Sanity check for train set. Each row must has only one 1.')
for i in range(len(labels_train)):
    for j in range(len(labels_train[str(i)])):
        if sum(labels_train[str(i)][j]) > 1 or sum(labels_train[str(i)][j]) == 0:
            print(i,j) 

#Task labels sanity check for test set. Each row must has only one '1' 
print('Sanity check for test set. Each row must has only one 1.')
for i in range(len(labels_test)):
    for j in range(len(labels_test[str(i)])):
        if sum(labels_test[str(i)][j]) > 1 or sum(labels_test[str(i)][j]) == 0:
            print(i,j)

#Task labels sanity check for train set. Each task label must has 1 at right location
print('Sanity check for train set. Each task has 1 at right location.')
for i in range(len(labels_train)):
    for j in range(len(labels_train[str(i)])):
        if sum(labels_train[str(i)][j][task_labels[i]]) != 1:
            print(i,j)
           
            
#Task labels sanity check for test set. Each task must has a non zero label
print('Sanity check for test set. Each task has 1 at right location.')
for i in range(len(labels_test)):
    for j in range(len(labels_test[str(i)])):
        if sum(labels_test[str(i)][j][task_labels[i]]) != 1:
            print(i,j)

Sanity check for train set. Each row must has only one 1.
Sanity check for test set. Each row must has only one 1.
Sanity check for train set. Each task has 1 at right location.
Sanity check for test set. Each task has 1 at right location.


In [4]:
# -*- coding: utf-8 -*-
import tensorflow as tf
import numpy as np
import datetime
from tensorflow.contrib.learn.python.learn.datasets.mnist import DataSet
from tensorflow.contrib.learn.python.learn.datasets import base
from tensorflow.python.framework import dtypes
from tensorflow.examples.tutorials.mnist import input_data
from OWMLayer_3Layers import OWMLayer
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # ignore warning
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # use gpu

# Parameters
# ==================================================
tf.flags.DEFINE_integer("num_checkpoints", 3, "Number of checkpoints to store (default: 5)")
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")
tf.app.flags.DEFINE_string("buckets", "", "")
tf.app.flags.DEFINE_string("checkpointDir", "", "oss info")
tf.flags.DEFINE_integer("num_class", 10, "")
tf.flags.DEFINE_integer("batch_size", 64, "Batch Size (default: 64)")
tf.flags.DEFINE_integer("epoch", 50, "")
FLAGS = tf.flags.FLAGS
# ==================================================

In [5]:
import pdb
import pprint
def next_batch(l, n,m): 
    final = [l[i * n:(i + 1) * n,:] for i in range((len(l) + n - 1) // n )]
    #Repeating the frazme segmentation for different epochs
    if m >= len(final): 
        m = m % len(final)
        if m == 0:
            np.random.shuffle(l)
            final = [l[i * n:(i + 1) * n,:] for i in range((len(l) + n - 1) // n )]
    return(final[m])       

In [8]:
def train(Task_list):
    # Training
    # ==================================================
    g1 = tf.Graph()
    middle1 = 60
    middle2 = 30
    with g1.as_default():
        OWM = OWMLayer([[43 + 1, middle1], [middle1 + 1, middle2], [middle2 + 1, 10]], seed_num=79)

    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    test_array = []
    with tf.Session(graph=g1, config=config) as sess1:
        # Initialize all variables
        init = [tf.global_variables_initializer(), tf.local_variables_initializer()]
        sess1.run(init)
        task_num = 5
        for j in range(0, task_num):
            print("Training Disjoint UNSW Task %d" % (j + 1))
            # Update the parameters

            # Update the parameters
            epoch_owm = FLAGS.epoch
            batch_size_owm = FLAGS.batch_size
            #print('Task performs classification between {0} and rest.'.format(set(np.argmax(mnist_list[j].train.labels,axis=1))))
            print('Task performs classification between {0}'.format(set(np.argmax(Task_list[1][str(j)],axis=1))))
            all_data = len(Task_list[1][str(j)][:])
            all_step = all_data*epoch_owm//batch_size_owm
            for current_step in range(all_step):
                lamda = current_step/all_step
                current_step = current_step+1
                batch_xs = next_batch(Task_list[0][str(j)][:], batch_size_owm, current_step)
                batch_ys = next_batch(Task_list[1][str(j)][:], batch_size_owm, current_step)
                feed_dict = {
                    OWM.input_x: batch_xs,
                    OWM.input_y: batch_ys,
                    OWM.lr_array: np.array([[0.02]]),
                    OWM.alpha_array: np.array([[0.9 * 0.001 ** lamda, 1.0 * 0.1 ** lamda, 0.6]]),
                }
                acc, loss,  _, = sess1.run([OWM.accuracy, OWM.loss, OWM.back_forward], feed_dict,)
                if current_step % (all_step // 2) == 0:
                    print("Train->>>Task: [{:d}/{:d}] Step: {:d}/{:d} Train: loss: {:.2f}, acc: {:.2f}  %"
                          .format(j+1, task_num, current_step*epoch_owm // all_step+1,
                                  epoch_owm, loss, acc * 100))
                if current_step % (all_step // 20) == 0:
                    feed_dict = {
                        OWM.input_x: Task_list[2][str(j)][:],
                        OWM.input_y: Task_list[3][str(j)][:],
                    }
                    acc, loss = sess1.run([OWM.accuracy, OWM.loss], feed_dict)
                    print("Eval->>> Task: [{:d}/{:d}] Step: [{:d}/{:d}] acc: {:g}  %"
                          .format(j+1, task_num, current_step*epoch_owm // all_step+1, epoch_owm, acc * 100))
            P1_mat = sess1.run(OWM.P1, feed_dict,)
            P2_mat = sess1.run(OWM.P2, feed_dict,)
            P3_mat = sess1.run(OWM.P3, feed_dict,)
            print('Shape of matrix P1 = {0}, P2 = {1} and P3 = {2}'.format(P1_mat.shape, P2_mat.shape, P3_mat.shape))
            print('Rank of P1 = {0}, P2= {1} and P3={2}'.format(np.linalg.matrix_rank(P1_mat), np.linalg.matrix_rank(P2_mat), np.linalg.matrix_rank(P3_mat)))        
            print("Test on Previous Datasets:")
            correct = []
            Per_task_acc_holder = {}
            for i_test in range(j + 1):
                feed_dict = {
                    OWM.input_x: Task_list[2][str(i_test)][:],
                    OWM.input_y: Task_list[3][str(i_test)][:],
                }
                accu, = sess1.run([OWM.accuracy], feed_dict)
                Per_task_acc_holder['Task ' + str(i_test+1)] = accu
                correct.append(accu)
            pprint.pprint(Per_task_acc_holder)
            test_accu = 100 * np.mean(correct)
            test_array.append([test_accu,Per_task_acc_holder])
            print("Aggregate Test Accuracy:->>>[{:d}/{:d}], acc: {:.2f} %".format(j + 1, task_num, test_accu))
        feed_dict = {
            OWM.input_x: X_test,
            OWM.input_y: Y_test_onehot_encoded,
        }
        accu, loss = sess1.run([OWM.accuracy, OWM.loss], feed_dict)
        print("accu_owm {:g} %\n".format(accu * 100))
        print('Aggregate test accuracy of trained tasks after completing each task trainng: {}'.format(test_array))


def main(_):
    # Create 10 disjoint MNIST datasets
    Task_list = [tasks_train, labels_train, tasks_test, labels_test]
    train(Task_list)

if __name__ == '__main__':
    tf.app.run()


Training Disjoint UNSW Task 1
Task performs classification between {4, 5}
Eval->>> Task: [1/5] Step: [3/50] acc: 68.7474  %
Eval->>> Task: [1/5] Step: [5/50] acc: 73.876  %
Eval->>> Task: [1/5] Step: [8/50] acc: 77.3632  %
Eval->>> Task: [1/5] Step: [10/50] acc: 81.3832  %
Eval->>> Task: [1/5] Step: [13/50] acc: 79.8553  %
Eval->>> Task: [1/5] Step: [15/50] acc: 81.3677  %
Eval->>> Task: [1/5] Step: [18/50] acc: 80.4482  %
Eval->>> Task: [1/5] Step: [20/50] acc: 78.7467  %
Eval->>> Task: [1/5] Step: [23/50] acc: 78.36  %
Eval->>> Task: [1/5] Step: [25/50] acc: 78.9186  %
Train->>>Task: [1/5] Step: 26/50 Train: loss: 0.57, acc: 78.12  %
Eval->>> Task: [1/5] Step: [28/50] acc: 78.6385  %
Eval->>> Task: [1/5] Step: [30/50] acc: 80.2643  %
Eval->>> Task: [1/5] Step: [33/50] acc: 33.2377  %
Eval->>> Task: [1/5] Step: [35/50] acc: 22.2415  %
Eval->>> Task: [1/5] Step: [38/50] acc: 21.3254  %
Eval->>> Task: [1/5] Step: [40/50] acc: 21.994  %
Eval->>> Task: [1/5] Step: [43/50] acc: 21.5231  %


SystemExit: 

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2918: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
